In [1]:
from sympy import *

init_printing(use_latex="mathjax")

In [2]:
def diff_total(expr, diff_by, diff_map):
    """
    Example:

        theta, theta_dot, t = symbols(r'theta \dot{\theta} t')
        diff_total(cos(theta) + t**2, t, {theta:theta_dot})

    returns

        -theta_dot*sin(theta) + 2*t
    """
    result = Integer(0)
    for var, var_diff in diff_map.items():
        result += diff(expr, var) * var_diff
    result += diff(expr, diff_by)
    return result

In [3]:
M = symbols(r'M', real=True, positive=True)
m1, l1, I1 = symbols(r'm_1 l_1 I_1', real=True, positive=True)
m2, l2, I2 = symbols(r'm_2 l_2 I_2', real=True, positive=True)
g = symbols(r'g')

t = symbols(r't')
x, x_dot, x_ddot = symbols(r'x \dot{x} \ddot{x}', real=True)
theta1, theta1_dot, theta1_ddot = symbols(r'\theta_1 \dot{\theta_1} \ddot{\theta_1}', real=True)
theta2, theta2_dot, theta2_ddot = symbols(r'\theta_2 \dot{\theta_2} \ddot{\theta_2}', real=True)

F = symbols(r'F', real=True)

diff_map = {
         x:      x_dot,      x_dot:      x_ddot,
    theta1: theta1_dot, theta1_dot: theta1_ddot, 
    theta2: theta2_dot, theta2_dot: theta2_ddot}

In [4]:
v_A = Matrix([x_dot, 0])

v_B = v_A + l1/2*theta1_dot * Matrix([cos(theta1), -sin(theta1)])

v_C = v_A + l1*theta1_dot * Matrix([cos(theta1), -sin(theta1)])
v_D = v_C + l2/2*theta2_dot * Matrix([cos(theta2), -sin(theta2)])


print('v_A')
display(v_A)
print('v_B')
display(v_B)
print('v_D')
display(v_D)

v_A


⎡\dot{x}⎤
⎢       ⎥
⎣   0   ⎦

v_B


⎡\dot{\theta_1}⋅l₁⋅cos(\theta₁)          ⎤
⎢────────────────────────────── + \dot{x}⎥
⎢              2                         ⎥
⎢                                        ⎥
⎢    -\dot{\theta_1}⋅l₁⋅sin(\theta₁)     ⎥
⎢    ────────────────────────────────    ⎥
⎣                   2                    ⎦

v_D


⎡                                 \dot{\theta_2}⋅l₂⋅cos(\theta₂)          ⎤
⎢\dot{\theta_1}⋅l₁⋅cos(\theta₁) + ────────────────────────────── + \dot{x}⎥
⎢                                               2                         ⎥
⎢                                                                         ⎥
⎢                                      \dot{\theta_2}⋅l₂⋅sin(\theta₂)     ⎥
⎢    -\dot{\theta_1}⋅l₁⋅sin(\theta₁) - ──────────────────────────────     ⎥
⎣                                                    2                    ⎦

In [5]:
# kinetic energy
T = M*v_A.norm()**2/2 + \
    m1*v_B.norm()**2/2 + I1*theta1_dot**2/2 + \
    m2*v_D.norm()**2/2 + I2*theta2_dot**2/2

display(Eq(symbols('T'), T))

                                                              ⎛              2
                                                              ⎜\dot{\theta_1} 
                     2                    2            2   m₁⋅⎜───────────────
    I₁⋅\dot{\theta_1}    I₂⋅\dot{\theta_2}    M⋅\dot{x}       ⎝               
T = ────────────────── + ────────────────── + ────────── + ───────────────────
            2                    2                2                           

   2    2                                                      2⎞      ⎛      
⋅l₁ ⋅sin (\theta₁)   ⎛\dot{\theta_1}⋅l₁⋅cos(\theta₁)          ⎞ ⎟      ⎜⎛     
────────────────── + ⎜────────────────────────────── + \dot{x}⎟ ⎟   m₂⋅⎜⎜\dot{
 4                   ⎝              2                         ⎠ ⎠      ⎝⎝     
───────────────────────────────────────────────────────────────── + ──────────
                      2                                                       

                                                  

In [6]:
# potential energy
V = m1*g*l1/2*cos(theta1) + m2*g*(l1*cos(theta1) + l2/2*cos(theta2))

display(Eq(symbols('V'), V))

    g⋅l₁⋅m₁⋅cos(\theta₁)        ⎛                  l₂⋅cos(\theta₂)⎞
V = ──────────────────── + g⋅m₂⋅⎜l₁⋅cos(\theta₁) + ───────────────⎟
             2                  ⎝                         2       ⎠

In [7]:
# Lagrange function 
L = T - V

In [8]:
expr1 = diff_total(diff(L, x_dot), t, diff_map) - diff(L, x)
expr1 = collect(simplify(expr1), [x_ddot, theta1_ddot, theta2_ddot, theta1_dot, theta2_dot, g])
eq1 = Eq(expr1, F)

display(eq1)

                                                                              
\ddot{\theta_1}⋅l₁⋅(m₁ + 2⋅m₂)⋅cos(\theta₁)   \ddot{\theta_2}⋅l₂⋅m₂⋅cos(\theta
─────────────────────────────────────────── + ────────────────────────────────
                     2                                        2               

                                            2                                 
₂)                            \dot{\theta_1} ⋅l₁⋅(m₁ + 2⋅m₂)⋅sin(\theta₁)   \d
── + \ddot{x}⋅(M + m₁ + m₂) - ─────────────────────────────────────────── - ──
                                                   2                          

            2                       
ot{\theta_2} ⋅l₂⋅m₂⋅sin(\theta₂)    
──────────────────────────────── = F
              2                     

In [9]:
expr2 = diff_total(diff(L, theta1_dot), t, diff_map) - diff(L, theta1)
expr2 = collect(simplify(expr2), [x_ddot, theta1_ddot, theta2_ddot, theta1_dot, theta2_dot, g])
eq2 = Eq(expr2, 0)

display(eq2)

                ⎛       2            ⎞                                        
                ⎜     l₁ ⋅m₁     2   ⎟   \ddot{\theta_2}⋅l₁⋅l₂⋅m₂⋅cos(\theta₁ 
\ddot{\theta_1}⋅⎜I₁ + ────── + l₁ ⋅m₂⎟ + ─────────────────────────────────────
                ⎝       4            ⎠                          2             

                                                                              
- \theta₂)            ⎛l₁⋅m₁⋅cos(\theta₁)                     ⎞   \dot{\theta_
────────── + \ddot{x}⋅⎜────────────────── + l₁⋅m₂⋅cos(\theta₁)⎟ + ────────────
                      ⎝        2                              ⎠               

  2                                                                           
2} ⋅l₁⋅l₂⋅m₂⋅sin(\theta₁ - \theta₂)     ⎛  l₁⋅m₁⋅sin(\theta₁)                 
─────────────────────────────────── + g⋅⎜- ────────────────── - l₁⋅m₂⋅sin(\the
           2                            ⎝          2                          

         
    ⎞    
ta₁)⎟ = 0
    ⎠    

In [10]:
expr3 = diff_total(diff(L, theta2_dot), t, diff_map) - diff(L, theta2)
expr3 = collect(simplify(expr3), [x_ddot, theta1_ddot, theta2_ddot, theta1_dot, theta2_dot, g])
eq3 = Eq(expr3, 0)

display(eq3)

                                                                  ⎛       2   
\ddot{\theta_1}⋅l₁⋅l₂⋅m₂⋅cos(\theta₁ - \theta₂)                   ⎜     l₂ ⋅m₂
─────────────────────────────────────────────── + \ddot{\theta_2}⋅⎜I₂ + ──────
                       2                                          ⎝       4   

⎞                                               2                             
⎟   \ddot{x}⋅l₂⋅m₂⋅cos(\theta₂)   \dot{\theta_1} ⋅l₁⋅l₂⋅m₂⋅sin(\theta₁ - \thet
⎟ + ─────────────────────────── - ────────────────────────────────────────────
⎠                2                                       2                    

                              
a₂)   g⋅l₂⋅m₂⋅sin(\theta₂)    
─── - ──────────────────── = 0
               2              